In [34]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 21: Allergen Assessment ---</h2><p>You reach the train's last stop and the closest you can get to your vacation island without getting wet. There aren't even any boats here, but nothing can stop you now: you build a raft. You just need a few days' worth of food for your journey.</p>
<p>You don't speak the local language, so you can't read any ingredients lists. However, sometimes, allergens are listed in a language you <em>do</em> understand. You should be able to use this information to determine which ingredient contains which allergen and <span title="I actually considered doing this once. I do not recommend it.">work out which foods are safe</span> to take with you on your trip.</p>
<p>You start by compiling a list of foods (your puzzle input), one food per line. Each line includes that food's <em>ingredients list</em> followed by some or all of the allergens the food contains.</p>
<p>Each allergen is found in exactly one ingredient. Each ingredient contains zero or one allergen. <em>Allergens aren't always marked</em>; when they're listed (as in <code>(contains nuts, shellfish)</code> after an ingredients list), the ingredient that contains each listed allergen will be <em>somewhere in the corresponding ingredients list</em>. However, even if an allergen isn't listed, the ingredient that contains that allergen could still be present: maybe they forgot to label it, or maybe it was labeled in a language you don't know.</p>
<p>For example, consider the following list of foods:</p>
<pre><code>mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
</code></pre>
<p>The first food in the list has four ingredients (written in a language you don't understand): <code>mxmxvkd</code>, <code>kfcds</code>, <code>sqjhc</code>, and <code>nhms</code>. While the food might contain other allergens, a few allergens the food definitely contains are listed afterward: <code>dairy</code> and <code>fish</code>.</p>
<p>The first step is to determine which ingredients <em>can't possibly</em> contain any of the allergens in any food in your list. In the above example, none of the ingredients <code>kfcds</code>, <code>nhms</code>, <code>sbzzf</code>, or <code>trh</code> can contain an allergen. Counting the number of times any of these ingredients appear in any ingredients list produces <em><code>5</code></em>: they all appear once each except <code>sbzzf</code>, which appears twice.</p>
<p>Determine which ingredients cannot possibly contain any of the allergens in your list. <em>How many times do any of those ingredients appear?</em></p>
</article>


In [35]:
from collections import Counter
from copy import copy, deepcopy
from functools import reduce
from pprint import pprint


example = """
mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
"""


def ingredients_without_allergens(s: str) -> int:
    allergens_possible_candidates = {}
    all_ingredients = Counter()

    for line in s.strip().splitlines():
        ingredients, allergens = re.split(r"\s*\(contains\s*", line)
        ingredients = re.split(r"\s+", ingredients)
        allergens = re.split(r"\s*,\s+", allergens[:-1])

        all_ingredients += Counter(ingredients)

        for allergen in allergens:
            if allergen not in allergens_possible_candidates:
                allergens_possible_candidates[allergen] = set(ingredients)
            else:
                allergens_possible_candidates[allergen] &= set(ingredients)

    non_allegergent_ingredients = reduce(
        lambda acc, el: acc | el, allergens_possible_candidates.values(), set()
    )

    return sum(
        v for k, v in all_ingredients.items() if k not in non_allegergent_ingredients
    )


assert ingredients_without_allergens(example) == 5

In [36]:
with open("../input/day21.txt") as f:
    puzzle = f.read()

# print(
#     f"Part I: {how_many_bag_colors_can_eventually_contain_at_least_one_shiny_gold_bag(puzzle)}"
# )

ingredients_without_allergens(puzzle)  # 192 too low

1958

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1958</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that you've isolated the inert ingredients, you should have enough information to figure out which ingredient contains which allergen.</p>
<p>In the above example:</p>
<ul>
<li><code>mxmxvkd</code> contains <code>dairy</code>.</li>
<li><code>sqjhc</code> contains <code>fish</code>.</li>
<li><code>fvjkl</code> contains <code>soy</code>.</li>
</ul>
<p>Arrange the ingredients <em>alphabetically by their allergen</em> and separate them by commas to produce your <em>canonical dangerous ingredient list</em>. (There should <em>not be any spaces</em> in your canonical dangerous ingredient list.) In the above example, this would be <em><code>mxmxvkd,sqjhc,fvjkl</code></em>.</p>
<p>Time to stock your raft with supplies. <em>What is your canonical dangerous ingredient list?</em></p>
</article>

</main>


In [37]:
from more_itertools import first, one


def get_allergens_possible_candidates(s):
    allergens_possible_candidates = {}

    for line in s.strip().splitlines():
        ingredients, allergens = re.split(r"\s*\(contains\s*", line)
        ingredients = re.split(r"\s+", ingredients)
        allergens = re.split(r"\s*,\s+", allergens[:-1])

        for allergen in allergens:
            if allergen not in allergens_possible_candidates:
                allergens_possible_candidates[allergen] = set(ingredients)
            else:
                allergens_possible_candidates[allergen] &= set(ingredients)

    return allergens_possible_candidates


def get_first_canonical_element(allergens_possible_candidates):
    alergen, cononical_element = first(
        ((k, v) for k, v in allergens_possible_candidates.items() if len(v) == 1),
        (None, None),
    )
    if cononical_element is None:
        return (None, None)
    return alergen, one(cononical_element)


def canonical_dangerous_ingredient_list(s: str) -> int:
    allergens_possible_candidates = get_allergens_possible_candidates(s)

    canonical_dangerous_ingredients = []
    cononical_alergen, cononical_ingredient = get_first_canonical_element(
        allergens_possible_candidates
    )

    while cononical_ingredient is not None:
        canonical_dangerous_ingredients.append(
            (cononical_alergen, cononical_ingredient)
        )
        del allergens_possible_candidates[cononical_alergen]

        for allergen in list(allergens_possible_candidates.keys()):
            allergens_possible_candidates[allergen].discard(cononical_ingredient)

        cononical_alergen, cononical_ingredient = get_first_canonical_element(
            allergens_possible_candidates
        )

    return ",".join(i for _, i in sorted(canonical_dangerous_ingredients))


assert canonical_dangerous_ingredient_list(example) == "mxmxvkd,sqjhc,fvjkl"

In [38]:
print(f"Part II: {canonical_dangerous_ingredient_list(puzzle)}")

Part II: xxscc,mjmqst,gzxnc,vvqj,trnnvn,gbcjqbm,dllbjr,nckqzsg


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>xxscc,mjmqst,gzxnc,vvqj,trnnvn,gbcjqbm,dllbjr,nckqzsg</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
